In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import sys
import os

sys.path.append('../')
from functions.geometry import angle_between
from functions.tracking import update_landmarks_dict

In [2]:
# remove all files in data/tmp_frames  directory
for file in os.listdir('../data/tmp_frames'):
    os.remove(f'../data/tmp_frames/{file}')

In [3]:
create_output_video = False
local_minimum_threshold = 0.005

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# inputs
seconds_to_skip_beginning = 30
seconds_to_skip_end = 15
path_video = '../data/IMG_1476.mov'

# output
out_filename = 'tmp.mp4'

# variables
cap = cv2.VideoCapture(path_video)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration_seconds = n_frames/fps
frames_to_skip_beginning = int(round(seconds_to_skip_beginning * fps))
frames_to_skip_end = int(round(seconds_to_skip_end * fps))

# Define the codec and create VideoWriter object
if create_output_video:
    fourcc = cv2.VideoWriter_fourcc(*'H264') # for streamlit compatibility
    out = cv2.VideoWriter(out_filename, fourcc, fps, (width,  height))

I0000 00:00:1735039657.084250  243654 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-20.2.48), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


#### Reding and preprocessing

In [4]:
frame_count = 0
pose_undetected = 0

landmarks_dict = {
    'left_hip': [],
    'left_knee': [],
    'left_ankle': [],
    'left_shoulder': [],
    'left_elbow': [],
    'left_wrist': [],
}

frame_list = []

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# is there a way to process it faster and not frame by frame?
# possibility to save the edited video with landmarks and stuff
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1

    if not ret:
        break
    
    if  frames_to_skip_beginning <= frame_count < (n_frames - frames_to_skip_end):

        # Convert the frame to RGB: why this conversion to writeable?
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process the frame with MediaPipe Pose
        results = pose.process(image)
        
        # Convert the frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            update_landmarks_dict(results,width, height, landmarks_dict)
            frame_list.append(frame_count - frames_to_skip_beginning + 1)

            # Display angle close to the knee
            #cv2.rectangle(image, (int(knee[0]), int(knee[1]) - 20), (int(knee[0]) + 40, int(knee[1]) + 10), (0, 0, 0), -1)
            #cv2.putText(image, f'{int(angle)}', (int(knee[0]) + 5, int(knee[1]) + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (256, 256, 256), 1, cv2.LINE_AA)
        else:
            pose_undetected += 1

        if create_output_video:
            out.write(image)

        # save the image as compressed jpg 
        cv2.imwrite(f'../data/tmp_frames/frame_{frame_count - frames_to_skip_beginning + 1}.jpg', image)

cv2.destroyAllWindows()
cv2.waitKey(1)  # without this, will not close properly
cap.release()
out.release()

W0000 00:00:1735039657.524652  243805 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735039657.615561  243810 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735039664.181918  243806 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [6]:
# create a dataframe with the data ankle, knee, hip
df_tracking = pd.DataFrame({
    'frame': frame_list,
    'hip': landmarks_dict['left_hip'],
    'knee': landmarks_dict['left_knee'],
    'ankle': landmarks_dict['left_ankle'],
    'shoulder': landmarks_dict['left_shoulder'],
    'elbow': landmarks_dict['left_elbow'],
    'wrist': landmarks_dict['left_wrist'],
})

# create a column True false if the left ankle is at the minimum height
min_ankle = max([ankle[1] for ankle in df_tracking['ankle']])
df_tracking['min_height_ankle'] = df_tracking['ankle'].apply(lambda x: abs(x[1] - min_ankle)/min_ankle < local_minimum_threshold)

# calculate the angle between the hip, knee and ankle
df_tracking['angle_knee'] = df_tracking.apply(lambda x: angle_between(x['knee'], x['hip'], x['ankle']), axis=1)
df_tracking['angle_shoulder'] = df_tracking.apply(lambda x: angle_between(x['shoulder'], x['hip'], x['elbow']), axis=1)
df_tracking['angle_elbow'] = df_tracking.apply(lambda x: angle_between(x['elbow'], x['shoulder'], x['wrist']), axis=1)
# todo
#df_tracking['angle_torso'] = df_tracking.apply(lambda x: angle_between(x['hip'], x['shoulder'], x['wrist']), axis=1)

df_tracking.to_csv('../data/tmp_tracking.csv', index=False)

In [7]:
df_tracking[df_tracking['min_height_ankle'] == True] 

,frame,hip,knee,ankle,shoulder,elbow,wrist,min_height_ankle,angle_knee,angle_shoulder,angle_elbow
47,48,"[231.7974042892456, 327.61653900146484]","[206.37598514556885, 392.8310012817383]","[208.50072383880615, 471.2788772583008]","[171.87297105789185, 260.4197311401367]","[140.35564184188843, 304.7487258911133]","[98.42156767845154, 332.4393844604492]",True,157.152076,77.138105,158.850619
48,49,"[231.77581787109375, 327.4437713623047]","[206.80927991867065, 393.38104248046875]","[212.2863507270813, 472.31483459472656]","[171.41642689704895, 260.46478271484375]","[140.3924524784088, 304.76898193359375]","[98.40939044952393, 332.48817443847656]",True,155.292005,77.025794,158.436266
419,420,"[233.40760946273804, 327.45147705078125]","[205.49630641937256, 393.3086395263672]","[209.11673069000244, 471.92588806152344]","[172.9934048652649, 259.57550048828125]","[141.44615292549133, 305.23632049560547]","[100.58881402015686, 331.84974670410156]",True,154.395323,76.312000,157.719950
467,468,"[233.3164358139038, 328.19713592529297]","[204.18989896774292, 393.5520935058594]","[205.0786542892456, 472.5377655029297]","[173.42402815818787, 260.3968811035156]","[141.6601824760437, 305.386905670166]","[102.09697723388672, 331.4665985107422]",True,155.334381,76.679116,158.615342
811,812,"[232.993905544281, 326.1819076538086]","[203.1061577796936, 391.90006256103516]","[212.03282833099365, 471.19598388671875]","[165.84853649139404, 271.8109130859375]","[150.12394666671753, 322.29347229003906]","[104.48463678359985, 334.35508728027344]",True,149.121605,68.302220,122.104801
903,904,"[232.6689076423645, 326.02771759033203]","[203.14660549163818, 393.62171173095703]","[205.42693376541138, 471.77024841308594]","[171.66082978248596, 263.240909576416]","[144.2336118221283, 310.6825256347656]","[98.91890287399292, 333.5285186767578]",True,154.734872,74.210121,146.788855
1047,1048,"[231.8442463874817, 327.39437103271484]","[204.4980525970459, 393.5301971435547]","[202.7980899810791, 471.5711212158203]","[171.44352793693542, 262.50864028930664]","[142.65884399414062, 305.9322738647461]","[99.9326491355896, 331.4214324951172]",True,158.783498,76.489423,154.358653
1191,1192,"[232.10963487625122, 324.4783020019531]","[203.78329753875732, 391.4531707763672]","[206.74634456634521, 471.7393112182617]","[172.92400002479553, 259.7665786743164]","[142.52588152885437, 303.80807876586914]","[101.16758108139038, 330.9889221191406]",True,154.960960,77.060211,157.927113
1289,1290,"[232.474844455719, 325.1422882080078]","[205.53578853607178, 392.8131866455078]","[212.1123504638672, 473.5189437866211]","[171.94236516952515, 260.63167572021484]","[140.53147673606873, 304.50469970703125]","[98.93996357917786, 331.4271926879883]",True,153.634398,78.778568,158.516047
